In [1]:
import numpy as np
import pandas as pd
import pygad
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import ListedColormap
import seaborn as sns
import plotly.graph_objects as go
from PIL import Image
import os
import glob

In [2]:
# Define the F6 function (Schaffer's F6 function)
def f6(x, y):
    numerator = np.sin(np.sqrt(x**2 + y**2))**2 - 0.5
    denominator = (1 + 0.001 * (x**2 + y**2))**2
    return 0.5 - numerator / denominator


In [3]:
# Convert binary chromosome to float values
def binary_to_float(gene, min_val=-100, max_val=100, num_bits=25):
    max_int = 2**num_bits - 1
    int_val = int("".join(str(i) for i in gene), 2)
    float_val = min_val + (max_val - min_val) * int_val / max_int
    return float_val


In [4]:
# Fitness function
def fitness_func(ga_instance, solution, solution_idx):
    x = binary_to_float(solution[:25])
    y = binary_to_float(solution[25:])
    return f6(x, y)

In [6]:
# Parameters
num_runs = 32
num_generations = 500
population_size = 500
gene_length = 25 * 2  # For x and y
mutation_rate = 0.01
crossover_rate = 0.8

In [13]:
# Prepare for storing results
all_results = []
best_solutions_history = []
final_fitnesses = []

population_history = []  # For storing population snapshots
best_solutions = []      # For storing best individuals
generations_data = []    # For storing fitness statistics

In [14]:
# Create directory for animation frames
os.makedirs('animation_frames', exist_ok=True)

In [20]:
def on_generation_store_best(ga_instance):
    global population_history, best_solutions, generations_data
    
    # 1. Store population snapshot (every N generations to save memory)
    if ga_instance.generations_completed % 5 == 0:  # Store every 5 generations
        population_history.append({
            'generation': ga_instance.generations_completed,
            'population': ga_instance.population.copy(),
            'fitness': ga_instance.last_generation_fitness.copy()
        })
    
    # 2. Track best solution (original functionality)
    best_solution = ga_instance.best_solution()
    best_solutions.append({
        'generation': ga_instance.generations_completed,
        'solution': best_solution[0],
        'fitness': best_solution[1]
    })
    
    # 3. Track generational statistics
    generations_data.append({
        'Generation': ga_instance.generations_completed,
        'Fitness': best_solution[1],
        'Run': run_number
    })
    
    # Optional: Print progress
    if ga_instance.generations_completed % 50 == 0:
        print(f"Run {run_number}, Gen {ga_instance.generations_completed}, Fitness: {best_solution[1]:.6f}")

In [16]:
# Modify the GA initialization to include custom population initialization
def init_population():
    # Create a population with more spread-out values
    population = []
    for _ in range(population_size):
        # Method 1: Force some individuals to be far from (0,0)
        if np.random.rand() < 0.8:  # 80% chance to be far from center
            # Generate values in [-100,-10] or [10,100]
            x_bits = np.random.choice([0,1], size=25)
            y_bits = np.random.choice([0,1], size=25)
            
            # Ensure they're not too close to zero
            x_val = binary_to_float(x_bits)
            y_val = binary_to_float(y_bits)
            while -10 < x_val < 10 or -10 < y_val < 10:
                x_bits = np.random.choice([0,1], size=25)
                y_bits = np.random.choice([0,1], size=25)
                x_val = binary_to_float(x_bits)
                y_val = binary_to_float(y_bits)
        else:  # 20% chance to be anywhere
            x_bits = np.random.choice([0,1], size=25)
            y_bits = np.random.choice([0,1], size=25)
        
        individual = np.concatenate([x_bits, y_bits])
        population.append(individual)
    
    return np.array(population)

In [21]:
# Run the genetic algorithm multiple times
for run in range(1, num_runs + 1):
    print(f"\nStarting run {run}/{num_runs}")
    
    # Reset tracking variables for each run
    generations_data = []
    best_solutions = []
    run_number = run
    
    # Initialize GA
    ga_instance = pygad.GA(
        num_generations=num_generations,
        num_parents_mating=population_size // 2,
        fitness_func=fitness_func,
        sol_per_pop=population_size,
        num_genes=gene_length,
        gene_type=int,
        gene_space=[0, 1],
        mutation_percent_genes=mutation_rate * 100,
        crossover_type="single_point",
        crossover_probability=crossover_rate,
        parent_selection_type="rws",  # Roulette Wheel Selection (Fitness Proportionate)
        #stop_criteria=["reach_1.0", f"saturate_{10}"],  # Stop if fitness reaches 1 or no improvement for 10 generations
        on_generation=on_generation_store_best,
        initial_population=init_population(),  # Add this line
        suppress_warnings=True
    )
    
    # Run GA
    ga_instance.run()
    
    # Store results
    solution, solution_fitness, _ = ga_instance.best_solution()
    final_fitnesses.append(solution_fitness)
    all_results.extend(generations_data)
    best_solutions_history.append(best_solutions)
    
    print(f"Run {run} completed. Best fitness: {solution_fitness:.6f}")


Starting run 1/32
Run 1, Gen 50, Fitness: 0.990284
Run 1, Gen 100, Fitness: 0.990284
Run 1, Gen 150, Fitness: 0.990284
Run 1, Gen 200, Fitness: 0.990284
Run 1, Gen 250, Fitness: 0.990284
Run 1, Gen 300, Fitness: 0.990284
Run 1, Gen 350, Fitness: 0.990284
Run 1, Gen 400, Fitness: 0.990284
Run 1, Gen 450, Fitness: 0.990284
Run 1, Gen 500, Fitness: 0.990284
Run 1 completed. Best fitness: 0.990284

Starting run 2/32
Run 2, Gen 50, Fitness: 0.990284
Run 2, Gen 100, Fitness: 0.990284
Run 2, Gen 150, Fitness: 0.990284
Run 2, Gen 200, Fitness: 0.990284
Run 2, Gen 250, Fitness: 0.990284
Run 2, Gen 300, Fitness: 0.990284
Run 2, Gen 350, Fitness: 0.990284
Run 2, Gen 400, Fitness: 0.990284
Run 2, Gen 450, Fitness: 0.990284
Run 2, Gen 500, Fitness: 0.990284
Run 2 completed. Best fitness: 0.990284

Starting run 3/32
Run 3, Gen 50, Fitness: 0.990283
Run 3, Gen 100, Fitness: 0.990284
Run 3, Gen 150, Fitness: 0.990284
Run 3, Gen 200, Fitness: 0.990284
Run 3, Gen 250, Fitness: 0.990284
Run 3, Gen 300, 

In [22]:
# Convert results to DataFrame
results_df = pd.DataFrame(all_results)

# Create animation for the best run
best_run_idx = np.argmax(final_fitnesses)
best_run_solutions = best_solutions_history[best_run_idx]

# Create heatmap of the function
x = np.linspace(-100, 100, 200)
y = np.linspace(-100, 100, 200)
X, Y = np.meshgrid(x, y)
Z = f6(X, Y)


In [23]:
# Generate frames for animation
frame_files = []
for i, gen_data in enumerate(best_run_solutions):
    plt.figure(figsize=(12, 6))
    plt.suptitle('Best run: Optimal solution along the generations')
    
    # Plot heatmap
    plt.subplot(1, 2, 1)
    plt.contourf(X, Y, Z, levels=50, cmap='viridis')
    plt.colorbar(label='F6 Value')
    
    # Plot best individual
    solution = gen_data['solution']
    x_val = binary_to_float(solution[:25])
    y_val = binary_to_float(solution[25:])
    plt.scatter(x_val, y_val, c='red', s=50, edgecolor='white')
    plt.title(f'Generation {gen_data["generation"]}\nFitness: {gen_data["fitness"]:.6f}')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim(-100, 100)
    plt.ylim(-100, 100)
    
    # Plot fitness trajectory
    plt.subplot(1, 2, 2)
    current_generations = [g['generation'] for g in best_run_solutions[:i+1]]
    current_fitnesses = [g['fitness'] for g in best_run_solutions[:i+1]]
    plt.plot(current_generations, current_fitnesses, 'b-')
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    plt.title('Fitness Progress')
    plt.ylim(0, 1.1)
    plt.grid(True)
    
    # Save frame
    frame_file = f'animation_frames/frame_{i:03d}.png'
    plt.savefig(frame_file, dpi=100, bbox_inches='tight')
    plt.close()
    frame_files.append(frame_file)

In [24]:
# Create GIF animation - MODIFIED VERSION
images = []
for f in frame_files:
    img = Image.open(f)
    # Convert all images to consistent mode (RGB) and size
    if img.mode == 'RGBA':
        img = img.convert('RGB')
    images.append(img)

# Ensure all images have same size by resizing to first image's dimensions
width, height = images[0].size
for i in range(1, len(images)):
    if images[i].size != (width, height):
        images[i] = images[i].resize((width, height))

images[0].save('f6_optimization.gif',
               save_all=True,
               append_images=images[1:],
               duration=100,
               loop=0)

# Clean up frame files
for f in frame_files:
    os.remove(f)

In [25]:
# Plot fitness over generations with average and standard deviation
plt.figure(figsize=(10, 6))

# Calculate statistics
max_generations = results_df['Generation'].max()
avg_fitness = results_df.groupby('Generation')['Fitness'].mean()
std_fitness = results_df.groupby('Generation')['Fitness'].std()

# Plot individual runs
#for run in range(1, num_runs + 1):
#    run_data = results_df[results_df['Run'] == run]
#    plt.plot(run_data['Generation'], run_data['Fitness'], alpha=0.2, color='blue')

# Plot average and standard deviation
plt.plot(avg_fitness.index, avg_fitness, 'b-', linewidth=2, label='Average Fitness')
plt.fill_between(avg_fitness.index,
                 avg_fitness - std_fitness,
                 avg_fitness + std_fitness,
                 color='gray', alpha=0.3, label='Standard Deviation')

plt.xlabel('Generation')
plt.ylabel('Fitness')
plt.title('Fitness Progress Across 32 Runs')
plt.legend()
plt.grid(True)
plt.savefig('fitness_progress.png', dpi=150, bbox_inches='tight')
plt.close()

# Population vector plot for best solution
best_run_ga = None
best_run_idx = np.argmax(final_fitnesses)
best_run_fitness = final_fitnesses[best_run_idx]


In [27]:
# Create population vector plot
#num_generations_recorded = len(population_history)
#if num_generations_recorded > 100:  # Limit to 100 generations for visualization
#    step = num_generations_recorded // 100
#    population_history = population_history[::step]
gen_100_data = [x for x in population_history if x['generation'] == 100][0]
population_at_100 = gen_100_data['population']
fitness_at_100 = gen_100_data['fitness']

population_matrix = np.array([population_at_100[0] for pop in population_history])  # Just show first individual for simplicity

plt.figure(figsize=(15, 8))
cmap = ListedColormap(['white', 'black'])
plt.imshow(population_matrix, cmap=cmap, aspect='auto', interpolation='none')
plt.xlabel('Gene Bit Position')
plt.ylabel('Generation')
plt.title(f'Population Vector (First Individual)\nBest Run Fitness: {best_run_fitness:.6f}')
plt.colorbar(ticks=[0, 1], label='Bit Value')
plt.savefig('population_vector.png', dpi=150, bbox_inches='tight')
plt.close()